In [ ]:
from platform import python_version
print(python_version())

### Crowdsource consensus (CSC)
  - The CSC is the consensus of Gemini MMC, PubMed answers, and Human consensus.
  - Each of these three sources holds equal weight in this composite measure.
  - Crowdsource consensus can be described as the consensus of the consensuses.

### 3 Sources ~ 2CRSP
  - Gemini MMC
  - PubMed answers
  - Human consensus

In [ ]:
import os, sys, pickle

import numpy as np
import pandas as pd
pd.set_option('display.width', 100)
pd.set_option('max_colwidth', 80)
pd.set_option("display.precision", 3)

import yaml

import seaborn as sns
sns.set_context("notebook", font_scale=1.4)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

sys.path.insert(1, '../src/')

from Basic import *
from biopax_lib import *
from gemini_lib import *
from pubmed_lib import *

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

with open('params.yml', 'r') as file:
    dic_yml=yaml.safe_load(file)

In [ ]:
root0=dic_yml['root0']
root_data_aux=dic_yml['root_data_aux']
email=dic_yml['email']

project=dic_yml['project']
s_project=dic_yml['s_project']

gene_protein=dic_yml['gene_protein']
s_omics=dic_yml['s_omics']

has_age=dic_yml['has_age']
has_gender=dic_yml['has_gender']

want_normalized=dic_yml['want_normalized']

abs_lfc_cutoff_inf=dic_yml['abs_lfc_cutoff_inf']
s_pathw_enrichm_method=dic_yml['s_pathw_enrichm_method']
num_min_degs_for_ptw_enr=dic_yml['num_min_degs_for_ptw_enr']

tolerance_pathway_index=dic_yml['tolerance_pathway_index']
type_sat_ptw_index=dic_yml['type_sat_ptw_index']
saturation_lfc_index=dic_yml['saturation_lfc_index']
chosen_model_sampling=dic_yml['chosen_model_sampling']

case_list=dic_yml['case_list']
case_sel_list=dic_yml['case_sel_list']

pval_pathway_cutoff=dic_yml['pval_pathway_cutoff']
fdr_pathway_cutoff=dic_yml['fdr_pathway_cutoff']
num_of_genes_cutoff=dic_yml['num_of_genes_cutoff']

run_list=dic_yml['run_list']
chosen_model_list=dic_yml['chosen_model_list']
i_dfp_list=dic_yml['i_dfp_list']

exp_normalization='quantile_norm' if want_normalized else None
normalization='not_normalized' if exp_normalization is None else exp_normalization

cfg=Config(project, s_project, case_list, root0)

case=case_list[0]

n_genes_annot_ptw, n_degs, n_degs_in_ptw, n_degs_not_in_ptw, degs_in_all_ratio=-1,-1,-1,-1,-1
abs_lfc_cutoff, fdr_lfc_cutoff, n_degs, n_degs_up, n_degs_dw=cfg.get_best_lfc_cutoff(case, 'not_normalized')

print(f"G/P LFC cutoffs: lfc={abs_lfc_cutoff:.3f}; fdr={fdr_lfc_cutoff:.3f}")
print(f"Pathway cutoffs: pval={pval_pathway_cutoff:.3f}; fdr={fdr_pathway_cutoff:.3f}; num of genes={num_of_genes_cutoff}")

In [ ]:
bpx=Biopax(gene_protein, s_omics, project, s_project, root0,
           case_list, has_age, has_gender, clone_objects=False,
           exp_normalization=exp_normalization, geneset_num=0, 
           num_min_degs_for_ptw_enr=num_min_degs_for_ptw_enr, 
           tolerance_pathway_index=tolerance_pathway_index, 
           s_pathw_enrichm_method=s_pathw_enrichm_method,
           abs_lfc_cutoff_inf=abs_lfc_cutoff_inf, 
           type_sat_ptw_index=type_sat_ptw_index, saturation_lfc_index=saturation_lfc_index)

case=case_list[0]

bpx.cfg.set_default_best_lfc_cutoff(normalization, abs_lfc_cutoff=1, fdr_lfc_cutoff=0.05)
ret, degs, degs_ensembl, dfdegs=bpx.open_case(case, verbose=False)
print("\nEcho Parameters:")
bpx.echo_parameters()

geneset_num=bpx.geneset_num

### MSD (multiple sources dataset): is_seldata=True - 2 cases randomly selected pathways (2CRSP)

In [ ]:
##########################
is_seldata=True
if is_seldata==True: i_dfp_list = [0]
i_dfp=0

case_list = case_sel_list
case_sel0 = case_sel_list[0]
case_sel1 = case_sel_list[1]

with_gender=bpx.has_gender
with_gender_list = [False, True] if with_gender else [False]

print(f"with_gender = {with_gender} because has_gender = {bpx.has_gender}")
##########################

In [ ]:
is_seldata, case_list, run_list, chosen_model_list, i_dfp_list, chosen_model_sampling

### Instantiating Gemini

In [ ]:
API_KEY=dic_yml['API_KEY']

disease=dic_yml['disease']
context_disease=dic_yml['context_disease']
n_sentences=dic_yml['n_sentences']
chosen_model_sampling=dic_yml['chosen_model_sampling']

gem=Gemini(bpx=bpx, is_seldata=is_seldata, disease=disease, context_disease=context_disease, 
           API_KEY=API_KEY, n_sentences=n_sentences, root0=root0, 
           chosen_model_list=chosen_model_list, i_dfp_list=i_dfp_list, chosen_model_sampling=chosen_model_sampling)
print("\n")
print(gem.disease, gem.is_seldata, gem.i_dfp_list, gem.chosen_model_list)
print("Context:", context_disease)

In [ ]:
gem.set_case(bpx.case, bpx.df_enr, bpx.df_enr0)

In [ ]:
gem.is_seldata, gem.bpx.case_list, gem.chosen_model_list, gem.i_dfp_list, gem.chosen_model_sampling

In [ ]:
prefix = s_project

terms1_param=dic_yml['terms1_param']
terms2_param=dic_yml['terms2_param']
terms_not_param=dic_yml['terms_not_param']
connective_param=dic_yml['connective_param']

remove_synonym_list=dic_yml['remove_synonym_list']
inidate=dic_yml['inidate']
enddate=dic_yml['enddate']

verbose_query=dic_yml['verbose_query']
force_query=dic_yml['force_query']

sleep_entrez=dic_yml['sleep_entrez']
retmax=dic_yml['retmax']

try_all_text=dic_yml['try_all_text']
text_quote=dic_yml['text_quote']
dec_ncpus=dic_yml['dec_ncpus']

sleep_TIKA=dic_yml['sleep_TIKA']
min_words_text=dic_yml['min_words_text']

print(f"prefix={prefix}, dates: {inidate} to {enddate}")
print(f"terms1: {terms1_param}")
print(f"terms_not: {terms_not_param}\n")

pub = Pubmed(bpx, gem, email, prefix, root0, 
             inidate=inidate, enddate=enddate, 
			 terms1_param=terms1_param, terms2_param=terms2_param, 
			 terms_not_param=terms_not_param, connective_param=connective_param,
             remove_synonym_list=remove_synonym_list, sleep_entrez=sleep_entrez, 
             retmax=retmax, try_all_text=True, text_quote='',
             root_data_aux=root_data_aux, dec_ncpus=dec_ncpus,
             sleep_TIKA=sleep_TIKA, min_words_text=min_words_text)

### Settings

In [ ]:
test=False
force=False
verbose=False

chosen_model=3
gemini_model=gem.gemini_models[chosen_model]
pub.gem.set_gemini_num_model(chosen_model, verbose=verbose)

# for selected data
query_type='strong'
N=30

case=case_sel0
print("")
dfsel = gem.open_yes_no_sampling(case=case, N=N, query_type=query_type, verbose=True)
print("")
dfsel.head(3)

In [ ]:
pub.gem.is_seldata, pub.gem.i_dfp_list, pub.gem.root_gemini0, case_list, with_gender_list

### Calc Gemini consensus counts

In [ ]:
pub.gem.chosen_model_list

In [ ]:
verbose=True
force=False

for run in run_list:
    print(">>>", run, end=' ')
    dfpiva = pub.gem.calc_gemini_dfpiva_all_models_one_run(run=run, case_list=case_list, 
                                                      chosen_model_list=chosen_model_list, 
                                                      force=force, verbose=verbose)
    if dfpiva is None:
        print("None")
    else:
        print(len(dfpiva))

print("---------------- end -----------------")

In [ ]:
verbose=True
run='run01'

dfpiva = gem.open_gemini_dfpiva_all_models_one_run(run=run,  chosen_model_list=chosen_model_list, verbose=verbose)
# print(dfpiva.columns)

print(len(dfpiva))
case = case_list[0]
df2 = dfpiva[dfpiva.case==case]
print(len(df2))
df2.head(3)

In [ ]:
case = case_list[1]
df2 = dfpiva[dfpiva.case==case]
print(len(df2))
df2.head(3)

In [ ]:
dfpiva.case.unique(), dfpiva.i_dfp.unique(), dfpiva.run.unique()

In [ ]:
verbose=False

run='run01'
case=case_list[0]
consensus='Yes'

dfpiv2 = pub.gem.open_gemini_consensus_counts_run_filter_idfp_consensus_run_all_models(run=run, i_dfp=i_dfp,
                                                                        chosen_model_list=chosen_model_list,
                                                                        consensus=consensus, verbose=verbose)
if dfpiv2 is None:
    dfpiv2 = pd.DataFrame()

print(">>>", run, i_dfp, case, "\n\n")

dfpiv3 = dfpiv2[dfpiv2.case == case]

print(len(dfpiv3))
dfpiv3.head(3)

In [ ]:
print("\n".join(dfpiv3.pathway))

In [ ]:
i=1
case=case_list[i]
print(">>>", case, "\n\n")

dfpiv3 = dfpiv2[dfpiv2.case == case]

print(len(dfpiv3))
dfpiv3.head(3)

### Reviewers

In [ ]:
# lib for Excel
# !pip3 install openpyxl --force

In [ ]:
pro = ['aparecida', 'fabio',  'kadu']
stu = ['vinicius', 'juliana', 'victor', 'gabriela', 'guilherme']

pub.set_researchers_students(pro=pro, stu=stu)

print(">>>", pub.reviewers, "\n\n")

group_names = [pro, stu]
group_list=['pro', 'stu']

### Merging reviewers

In [ ]:
verbose=True
force=False

dic = pub.merge_reviewers_answers(case_list=case_sel_list, 
                                  s_start='sampling_case_', query_type='query_type_strong_', force=force, verbose=verbose)
print(len(dic))
cases = list(dic.keys())
";".join(cases)

In [ ]:
dic[case_sel0].head(4)

In [ ]:
dic[case_sel1].head(4)

In [ ]:
case0 = case_sel0
dff1 = pub.open_reviewers_answers(case0, verbose=True)
print(len(dff1))
dff1.head(4)

In [ ]:
case0 = case_sel1
dff2 = pub.open_reviewers_answers(case0, verbose=True)
print(len(dff2))
dff2.head(4)

### Reviewers per group

In [ ]:
verbose=False
force=False

dfsf = pub.compare_2_reviewer_groups(group_names=group_names, group_list=group_list, case_list=case_sel_list,
                                     force=force, verbose=verbose)

pd.options.display.float_format = '{:,.3f}'.format

cols = ['group', 'case', 'n', 'perc_consensus_yes', 'perc_consensus_no', 'perc_consensus_doubt',
        'n_consensus_yes', 'n_consensus_no', 'n_consensus_doubt', 'pathways_yes', 'pathways_no',
        'pathways_doubt', 'researchers']
cols = ['group', 'case', 'n', 'perc_consensus_yes', 'perc_consensus_no', 'perc_consensus_doubt',
        'n_consensus_yes', 'n_consensus_no', 'n_consensus_doubt']

dfsf[cols]

### Reviewers consensus

In [ ]:
verbose=False
df1 = pub.open_reviewers_answers_group_case(group='pro', case=case_sel0, verbose=True)
df1.head(3)

In [ ]:
df2 = pub.open_reviewers_answers_group_case(group='pro', case=case_sel1, verbose=verbose)
df2.head(3)

In [ ]:
df3 = pub.open_reviewers_answers_group_case(group='stu', case=case_sel0, verbose=verbose)
df3.head(3)

In [ ]:
df4 = pub.open_reviewers_answers_group_case(group='stu', case=case_sel1, verbose=verbose)
df4.head(3)

### Run with_gender False - Medulloblastoma
#### calc_all_gemini_x_reviewers_agreement

In [ ]:
verbose=True
force=False

dic_gem = pub.calc_all_gemini_x_reviewers_agreement(run_list=run_list, case_list=case_list,  
                                                    chosen_model_list=chosen_model_list, 
                                                    force=force, verbose=verbose)

In [ ]:
dic_gem.keys()

In [ ]:
keys = list(dic_gem.keys())

key = keys[0]
print("Agreement with Gemini (consensus)", key)

dfa = dic_gem[key]

cols1 = ['run', 'case', 'person', 'n_total', 'n_yes_gemini_cons', 'n_no_gemini_cons', 'n_agree',
         'n_agree_yes', 'n_agree_no', 'agree', 'agree_std', 'agree_yes', 'agree_std_yes', 'agree_no',
         'agree_std_no', 'perc_agree', 'perc_agree_yes', 'perc_agree_no', 'rank']

cols1 = ['person', 'case', 'n_total', 'n_yes_gemini_cons', 'n_no_gemini_cons', 'n_agree',
         'n_agree_yes', 'n_agree_no', 'agree', 'agree_std', 'agree_yes', 'agree_no', 'rank']
dfa = dfa[cols1]

cols2 = ['person', 'case', 'n_total', 'n_yes', 'n_no', 'n_agree', 'n_agree_yes', 'n_agree_no',
         'agree', 'agree_std', 'agree_yes', 'agree_no', 'rank']
dfa.columns = cols2

def style_060(v, props=''):
    if isinstance(v, float):
        return props if v >= 0.6 else None
    return None

def style_030(v, props=''):
    if isinstance(v, float):
        return props if v < 0.3 else None
    return None

s2 = dfa.style.map(style_060, props='color:cyan;').\
               map(style_030, props='color:tomato;').\
               map(lambda v: None if not isinstance(v, float) else ('opacity: 70%;' if (v < 0.2) and (v > -0.2) else None))
s2

In [ ]:
key = keys[1]
print("Agreement with Gemini (consensus)", key)
dfa = dic_gem[key]

dfa = dic_gem[key]
dfa = dfa[cols1]
dfa.columns = cols2

s2 = dfa.style.map(style_060, props='color:cyan;').\
               map(style_030, props='color:tomato;').\
               map(lambda v: None if not isinstance(v, float) else ('opacity: 70%;' if (v < 0.2) and (v > -0.2) else None))
s2

#### PubMed agreements
  - rank: classification
  - n_total: number of pathways
  - n_agree: number of pathways that the reviewer agree with PubMed (Yes x Yes, No x No)
    - n_agree_yes: number of pathways that the reviewer agree with PubMed Yes x Yes
    - n_agree_no: number of pathways that the reviewer agree with PubMed No x No

In [ ]:
verbose=False
force=False

dic_pub = pub.calc_all_pubmed_x_reviewers_agreement(case_list=case_list, i_dfp_list=i_dfp_list, 
                                                    with_gender_list=with_gender_list, force=force, verbose=verbose)

In [ ]:
dic_pub.keys()

In [ ]:
keys = list(dic_pub.keys())

key = keys[0]
print("Agreement with PubMed", key)
df_wnt = dic_pub[key]

cols1 = ['person', 'case', 'n_total', 'n_yes_pub_has_pmid', 'n_no_pub_has_pmid', 'n_agree',
         'n_agree_yes', 'n_agree_no', 'agree', 'agree_std', 'agree_yes', 'agree_std_yes', 'agree_no',
         'agree_std_no', 'perc_agree', 'perc_agree_yes', 'perc_agree_no', 'rank']

cols1 = ['person', 'case', 'n_total', 'n_yes_pub_has_pmid', 'n_no_pub_has_pmid', 'n_agree',
         'n_agree_yes', 'n_agree_no', 'agree', 'agree_std', 'agree_yes', 'agree_no', 'rank']
df_wnt = df_wnt[cols1]

cols2 = ['person', 'case', 'n_total', 'n_yes', 'n_no', 'n_agree', 'n_agree_yes', 'n_agree_no',
         'agree', 'agree_std', 'agree_yes', 'agree_no', 'rank']
df_wnt.columns = cols2


dfsty = df_wnt.style.map(style_060, props='color:cyan;').\
                     map(style_030, props='color:tomato;').\
                     map(lambda v: None if not isinstance(v, float) else ('opacity: 70%;' if (v < 0.2) and (v > -0.2) else None))
dfsty

In [ ]:
key = keys[1]
print("Agreement with PubMed", key)
df_g4 = dic_pub[key]

df_g4 = df_g4[cols1]
df_g4.columns = cols2

dfsty = df_g4.style.map(style_060, props='color:cyan;').\
                    map(style_030, props='color:tomato;').\
                    map(lambda v: None if not isinstance(v, float) else ('opacity: 70%;' if (v < 0.2) and (v > -0.2) else None))
dfsty

### Statistics PubMed WNT x G4

In [ ]:
s_stat, stat, pvalue, dof, expected = pub.calc_similarity_false_true(df_wnt, df_g4)
s_stat

### Merge all 3 sources and calc crowdsource: without gender is MB

In [ ]:
verbose=False
force=False
person_list = pub.pro + pub.stu

pub.merge_all_3sources_calc_crowd(person_list=person_list, run_list=run_list, case_list=case_list, 
                                  with_gender_list=with_gender_list, force=force, verbose=verbose)

#### open_gemini_agreement_person()

In [ ]:
verbose=True

run='run01'
case = case_list[0]

person = person_list[4]
print(">>>", person)

dfrev_gem = pub.open_gemini_agreement_person(run=run, case=case, person=person, verbose=verbose)
print(len(dfrev_gem))
dfrev_gem.head(3)

In [ ]:
case = case_list[1]

dfrev_gem = pub.open_gemini_agreement_person(run=run, case=case, person=person, verbose=verbose)
print(len(dfrev_gem))
dfrev_gem.head(3)

#### open_pubmed_agreement()

In [ ]:
case = case_list[0]

dfrev_pub = pub.open_pubmed_agreement(with_gender=with_gender, case=case, person=person, verbose=verbose)
print(len(dfrev_pub))
dfrev_pub.head(3)

In [ ]:
case = case_list[1]

dfrev_pub = pub.open_pubmed_agreement(with_gender=with_gender, case=case, person=person, verbose=verbose)
print(len(dfrev_pub))
dfrev_pub.head(3)

### Crowdsource merge 3 sources: return dfcrowd

In [ ]:
verbose=True
force=False
run='run01'
case=case_list[0]

dfcrowd = pub.merge_3sources_calc_crowd(with_gender=with_gender, person_list=person_list, run=run, 
                                        case=case, force=force, verbose=verbose)
dfcrowd.head(3).T

In [ ]:
case=case_list[1]

cols = ['case', 'run', 'with_gender', 'pathway_id', 'pathway', 'i_dfp', 'n_pmid', 'pub_has_pmid',  'gem_consensus',
       'n_yes_gem', 'agree_pubgem', 'rev_consensus', 'rev_consensus_n_yes',
       'rev_consensus_n_no', 'unanimous_reviewers']

dfcrowd = pub.merge_3sources_calc_crowd(with_gender=with_gender, person_list=person_list, run=run, 
                                        case=case, force=False, verbose=verbose)
print(len(dfcrowd))
dfcrowd[cols].head(3).T

### Crowd and 3Sources Agreements

  - count_all_crowd_agreements()
    - count_crowd_agreements()
      - return dfcrowd
      - get all agrees and transform in accuracies

In [ ]:
verbose=False
force=False

pub.count_all_crowd_agreements(with_gender_list=with_gender_list, person_list=person_list,
                               run_list=run_list, case_list=case_list, force=force, verbose=verbose)

In [ ]:
verbose=False

for run in run_list:
    print(">>>", run)
    for case in case_list:
        df_count = pub.count_crowd_agreements(with_gender=with_gender, person_list=person_list, run=run, case=case, verbose=verbose)
        txt = f"\tcase {case:15} <crowd> {100*df_count.crowd_yes.mean():.1f}% <pubmed> {100*df_count.pubmed_has.mean():.1f}%, " + \
              f"<gemini> {100*df_count.gemini_yes.mean():.1f}%, and <reviewers> {100*df_count.review_yes.mean():.1f}%"
        print(txt)

### Accuracies
  - calc_all_crowd_accuracies()
    - loop run, case, with_gender
      - dic2_calc_crowd_accuracies(receive dic2)
    - calc: df_accu

In [ ]:
verbose=False
force=False

print(">>> person_list:", person_list, '\n')

all_text, dfall_crowd, df_accu = pub.calc_all_crowd_accuracies(person_list=person_list, run_list=run_list,
                                                               case_list=case_list, with_gender_list=with_gender_list, 
                                                               force=force, verbose=verbose)

len(dfall_crowd), dfall_crowd.run.unique(), dfall_crowd.case.unique(), dfall_crowd.with_gender.unique()

In [ ]:
dfall_crowd.groupby(['run', 'case', 'with_gender']).count().reset_index().iloc[:,:4]

#### Crowdsource summary text

In [ ]:
print(all_text)

### 3 sources accuracy
  - Pathways_crowd: how many pathways have Yes as responses
  - 50% of the pathways of the 2CRSP were Yes and 50% No
    - perc_pathws_crowd ~ 53%
    - perc_pathws_pubmed ~ 40%
    - perc_pathws_gemini ~ 66.7%
    - perc_pathws_reviewers ~ 53.3%

In [ ]:
df_accu.T

In [ ]:
run='run01'

for case in case_list:
    row = df_accu[ (df_accu.run == run) & (df_accu.case == case)].iloc[0]
    accu_gemini     = row.accu_gemini
    std_accu_gemini = row.std_accu_gemini
    accu_pubmed     = row.accu_pubmed
    std_accu_pubmed = row.std_accu_pubmed
    accu_review     = row.accu_review
    std_accu_review = row.std_accu_review

    msg  = f"for {run} case {case} the accuracies are:\n"
    msg += f"\tGemini    {100*accu_gemini:.1f}% ({100*std_accu_gemini:.1f}%)\n"
    msg += f"\tPubMed    {100*accu_pubmed:.1f}% ({100*std_accu_pubmed:.1f}%)\n"
    msg += f"\tReviewers {100*accu_review:.1f}% ({100*std_accu_review:.1f}%)\n\n"
    print(msg)
              

In [ ]:
df_count = pub.count_crowd_agreements(with_gender=with_gender, person_list=person_list, run=run, case=case, verbose=verbose)
print(len(df_count))
df_count.head(3)

In [ ]:
df_accu.columns

In [ ]:
all_text, dfall_crowd, df_accu = pub.calc_all_crowd_accuracies(person_list=person_list, run_list=run_list,
                                                               case_list=case_list, with_gender_list=with_gender_list, 
                                                               force=False, verbose=verbose)

In [ ]:
[x for x in dfall_crowd.columns if 'agree' in x]

In [ ]:
df_accu.columns

In [ ]:
[x for x in df_accu.columns if 'perc' in x]

### Comparing 3Sources

In [ ]:
verbose=True
force=False

dfbinv = pub.stat_3sources_binary_vars(run_list=run_list, case_list=case_list,
                                       with_gender_list=with_gender_list, person_list=person_list,
                                       force=force, verbose=verbose)

run='run01'
df2 = dfbinv[(dfbinv.run==run) & (dfbinv.with_gender==with_gender)]
df2

#### Significative

In [ ]:
df2[df2.fdr < 0.05]

#### CSC is only one: by gender False for MB
  - therefore, gemini accuracy is only one
  - PubMed accuracy is only one for MB

In [ ]:
run='run01'
dfbinv = pub.dfbinv
case=case_list[1]

df2 = dfbinv[(dfbinv.run==run) & (dfbinv.case==case) & (dfbinv.with_gender==with_gender)]
print(len(df2))
df2

In [ ]:
pub.dfbinv['compare'].unique()

In [ ]:
run='run01'

case = case_list[0]
compare = 'Accu_PubxGem'
pval1 = pub.which_stat_accuray(run=run, case=case, with_gender=with_gender, compare=compare)
compare = 'Accu_RevxGem'
pval2 = pub.which_stat_accuray(run=run, case=case, with_gender=with_gender, compare=compare)
pval1, pval2

In [ ]:
run='run01'
verbose=True
force=False

for which in ['reviewer', 'pathway', 'accuracy']:
    for with_gender in with_gender_list:
        fig, df = pub.barplot_accuracy(run=run, which=which, with_gender=with_gender, 
                                       with_gender_list=with_gender_list, person_list=person_list,
                                       run_list=run_list, case_list=case_list,
                                       chosen_model_list=chosen_model_list,
                                       width=1100, height=600, 
                                       fontsize=18, fontcolor='black',
                                       margin=dict( l=20, r=20, b=100, t=120, pad=4),
                                       plot_bgcolor="whitesmoke",
                                       minus_case=-0.15, minus_test=-0.10, minus_mnem=-0.05, 
                                       annot_fontfamily="Arial, monospace", annot_fontsize=16, 
                                       annot_fontcolor='black', savePlot=True, force=force, verbose=False)
        
        fig.show()

In [ ]:
df

In [ ]:
run='run01'
case=case_list[0]

dfa = df_accu[(df_accu.run == run) & (df_accu.with_gender==with_gender) &  (df_accu.case == case)]

row = dfa.iloc[0]
n = row.n

review_x_gemini = int(np.round(row.perc_review_x_gemini * n))
review_x_pubmed = int(np.round(row.perc_review_x_pubmed * n))

vals0 = [review_x_gemini, n-review_x_gemini]
vals1 = [review_x_pubmed, n-review_x_pubmed]

s_stat, stat, pvalue, _, _ = chi2_or_fisher_exact_test(vals0, vals1)
print(f"with_gender {with_gender} stats: {s_stat} - vals0 {vals0}, vals1 {vals1}")
print(f"perc_review_x_gemini {100*row.perc_review_x_gemini:.1f}%, perc_review_pubmed {100*row.perc_review_x_pubmed:.1f}%")
print(f"review_x_gemini {review_x_gemini}, review_x_pubmed {review_x_pubmed}")

 ### Accuracy run01 without gender

In [ ]:
cols=['case', 'perc_pathws_crowd', 'perc_pathws_pubmed',
      'perc_pathws_gemini', 'perc_pathws_review', 'accu_pubmed', 'accu_gemini',
      'accu_review']

df_accu[ (df_accu.run=='run01') & (df_accu.with_gender==with_gender)][cols].T

In [ ]:
df_accu

In [ ]:
df2 = df_accu[(df_accu.run == run) & (df_accu.case == case) & (df_accu.with_gender == with_gender)]
df2.T

### List reviewers' files

In [ ]:
term = case_sel0
files = [x for x in os.listdir(pub.root_curation) if term in x]
print(len(files))
print("\n".join(files[:5]))

In [ ]:
term = case_sel1
files = [x for x in os.listdir(pub.root_curation) if term in x]
print(len(files))
print("\n".join(files[:5]))

### Gemini's answers for run, case, model

In [ ]:
verbose=True

case=case_sel0
run='run01'
chosen_model=3
query_type='_strong'
want_pubmed=True

dfgem = pub.gem.get_gemini_results_by_case_model_semantics(run=run, case=case, chosen_model=chosen_model,
                                                           i_dfp_list=i_dfp_list, want_pubmed=want_pubmed, 
                                                           query_type=query_type, verbose=verbose)
if dfgem is None:
    dfgem = pd.DataFrame()
    
print(len(dfgem))
dfgem.head(3)

### Gemini x PubMed x Reviewers

In [ ]:
pub.pro, pub.stu, person_list

#### Reviewer x PubMed (with_gender)

In [ ]:
verbose=True

case = case_list[0]
person = person_list[0]

dfrev_pub = pub.open_pubmed_agreement(with_gender=with_gender, case=case, person=person, verbose=verbose)
print(len(dfrev_pub))
dfrev_pub.head(3)

#### Gemini x Reviewers

In [ ]:
verbose=True

run='run01'
case = case_list[0]
person = person_list[0]

dfrev_gem = pub.open_gemini_agreement_person(run=run, case=case, person=person, verbose=verbose)
print(len(dfrev_gem))
dfrev_gem.head(3)

### Gemini counts
 - must find 30 pathways (selected) - Ok

In [ ]:
force=False
verbose=True
chosen_model=3
run = 'run01'

dfall = gem.gemini_calc_answers_counts(run=run, case_list=case_list, chosen_model=chosen_model, force=force, verbose=verbose)
print(len(dfall))
dfall.head(3)

In [ ]:
case = case_sel0

dfall2 = dfall[dfall.case == case]
print(len(dfall2))
dfall2

In [ ]:
case = case_sel1

dfall2 = dfall[dfall.case == case]
print(len(dfall2))
dfall2

#### Gemini x PubMed

In [ ]:
verbose=False

dfc, dfg =pub.agreements_between_pubmed_and_gemini(run_list=run_list, case_list=case_list, i_dfp_list=i_dfp_list, 
                                                   chosen_model_list=chosen_model_list, with_gender_list=with_gender_list,
                                                   force=False, verbose=verbose)

print(len(dfc))

In [ ]:
dfc.run.unique()

In [ ]:
run='run01'

dfc2 = dfc[  (dfc.run==run) ]
dfc2

In [ ]:
run='run02'

dfc2 = dfc[ (dfc.run==run) ]
dfc2

In [ ]:
dfg

In [ ]:
run='run01'
case=case_list[0]

dfn, df_both, df_only_pubmed, df_only_gemini, mu, std, n, text = \
    pub.open_compare_pubmed_x_gemini(run=run, case=case, i_dfp=i_dfp, 
                                     with_gender=with_gender, 
                                     chosen_model_list=chosen_model_list, verbose=True)

dfn.head(3)

In [ ]:
mu =dfn.agree.mean()
std=dfn.agree.std()
      
f"with_gender {with_gender}: agree = {100*mu:.1f}% ({100*std:.1f}%)"